In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.foldseek import (
    get_foldseek_onehot,
    get_3di_sequences_from_file,
    get_3di_sequences_from_memory,
)
from src.data.utils import extract_dataset_information, replace_coordinates_in_pdb
from datasets import load_dataset
from datasets import Dataset, ClassLabel, Value, Image
import pickle
import pyarrow as pa
import h5py
from dataclasses import InitVar, dataclass, field, fields

from datasets.utils.file_utils import xopen
from io import BytesIO
import multiprocessing

import warnings
from Bio import BiopythonWarning

warnings.filterwarnings("ignore", category=BiopythonWarning)

In [5]:
md_cath_dataset = load_dataset('compsciencelab/mdCATH', split='train', streaming=True)
md_cath_dataset = md_cath_dataset.cast_column("image", Image(decode=False))

Resolving data files:   0%|          | 0/5400 [00:00<?, ?it/s]

{'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'num_taken': 8, 'ex_iterable': {'shard_idx': 7, 'shard_example_idx': 1}}}}}}}}}}}}


In [23]:
with open('../data/3Di/_state_dict.pkl', 'rb') as f:
    state_dict = pickle.load(f)
md_cath_dataset.load_state_dict(state_dict)
print(state_dict)

# md_cath_dataset = md_cath_dataset.skip(state_dict['num_taken'])

def process_mdcath_from_hf(value: dict):
    with xopen(value['image']['path'], "rb") as file:
        bytes_ = BytesIO(file.read())
        
    data = extract_dataset_information(bytes_)
    
    items = []
    for x in range(0, len(data["coords"])):
        new = replace_coordinates_in_pdb(
            original_pdb=data["pdb"], new_coordinates=data["coords"][x]
        )
        items.append(new)
        
    file_path = f"../data/3Di/{data['name']}.fasta"

    with open(file_path, "w") as file:
        file.write(f">{data['name']}|{data['seq']}\n" + "\n".join(get_3di_sequences_from_memory(pdb_files=items)))
    print(data['name'])
    return value

# md_cath_dataset_mapped = md_cath_dataset.map(process_mdcath_from_hf, num_proc=8)


pool = multiprocessing.Pool(processes=8)
md_cath_dataset_mapped = pool.map(process_mdcath_from_hf, md_cath_dataset)
pool.close()
pool.join()


state_dict = md_cath_dataset.state_dict()
with open('../data/3Di/_state_dict.pkl', 'wb') as f:
    pickle.dump(state_dict, f)
print(state_dict)


"""
1a02F00
153lA00
16pkA02
12asA00
1a0aA00
1a0hA01
1a0rP01
1a05A00
"""


"""
1a02F00
16pkA02
153lA00
1a0aA00
12asA00
1a0hA01
1a0rP01
1a05A00
"""

KeyError: 'num_taken'

In [ ]:
# @dataclass
# class HDF5:
#     decode: bool = True
    
#     def __repr__(self):
#         return 'dict'
    
#     def decode_example(self, value: dict, token_per_repo_id=None):
#         print('decoding...')
#         return 'decoded example'

# md_cath_dataset = md_cath_dataset.cast_column("image", HDF5())
# md_cath_dataset = md_cath_dataset.rename_column("image", "hdf5")


# item_name = "mdcath_dataset_1avyB00.h5"
# matching_item = md_cath_dataset.filter(lambda example: item_name in example['file_name'])



# md_cath_dataset = md_cath_dataset.cast_column("image", Image(decode=False))
# md_cath_dataset = md_cath_dataset.cast_column("image", dict)
# md_cath_dataset = md_cath_dataset.rename_column("image", "mdcath")




# item = item.cast_column("image", Image(decode=False))
# md_cath_dataset = md_cath_dataset.rename_column("image", "mdcath")


# item.info.builder_name= None


# ds_item = next(iter(md_cath_dataset))



# md_cath_dataset.info



# "mdcath_dataset_1avyB00.h5"

---

In [ ]:
# # md_cath_dataset = Dataset.from_dict({"a": range(10)}).to_iterable_dataset(num_shards=3)

# for idx, data in enumerate(md_cath_dataset):
#     data = h5py.File(data, 'r')
#     print(data.keys())
#     # data = extract_dataset_information(
#     #     file_path=f"../data/mdcath/mdcath_dataset_{item}.h5",
#     # )

#     # items = []
#     # for x in range(0, len(data[item]["coords"])):
#     #     new = replace_coordinates_in_pdb(
#     #         original_pdb=data[item]["pdb"], new_coordinates=data[item]["coords"][x]
#     #     )
#     #     items.append(new)
        
#     # file_path = f"../data/3Di/{item}.fasta"

#     # with open(file_path, "w") as file:
#     #     file.write(f">{item}|{data[item]['seq']}\n" + "\n".join(get_3di_sequences_from_memory(pdb_files=items)))
    
#     if idx == 0:
#         state_dict = md_cath_dataset.state_dict()
#         break


# with open('../data/3Di/_state_dict.pkl', 'wb') as f:
#     pickle.dump(state_dict, f)

In [ ]:
# state_dict

In [ ]:
# ds_item = next(iter(md_cath_dataset))